# --- Day 7: Bridge Repair ---

The Historians take you to a familiar rope bridge over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole all the operators from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

### Example:

```
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
```

Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are always evaluated left-to-right, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: add (`+`) and multiply (`*`).

### Example Analysis:

Only three of the above equations can be made true by inserting operators:

1. **`190: 10 19`**
   - There is only one position for an operator: between 10 and 19.
   - Choosing `+` gives 29, which is incorrect.
   - Choosing `*` gives 190, which matches the test value.

2. **`3267: 81 40 27`**
   - There are two positions for operators.
   - The four possible configurations of the operators are:
     - `81 + 40 + 27 = 148` (incorrect)
     - `81 + 40 * 27 = 3267` (correct)
     - `81 * 40 + 27 = 3267` (correct)
     - `81 * 40 * 27 = 87480` (incorrect)
   - Two configurations match the test value.

3. **`292: 11 6 16 20`**
   - There are three positions for operators.
   - The only configuration that works is: `11 + 6 * 16 + 20 = 292`.

The other equations cannot be solved using the available operators.

### Result:

The engineers just need the total calibration result, which is the sum of the test values from the equations that could possibly be true. In the above example:

- True equations: `190`, `3267`, `292`
- Total calibration result: `190 + 3267 + 292 = 3749`

---

### Your Task:

Determine which equations could possibly be true. What is their total calibration result?

In [50]:
from typing import Type
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from itertools import product


class Operator(ABC):
    @abstractmethod
    def __call__(self, a: int, b: int) -> int:
        pass


class Add(Operator):
    def __call__(self, a: int, b: int) -> int:
        return a + b


class Multiply(Operator):
    def __call__(self, a: int, b: int) -> int:
        return a * b


@dataclass
class Equation:
    value: int
    numbers: list[int]
    op_types: list[Type[Operator]] = field(default_factory=list)

    @staticmethod
    def evaluate_l2r(numbers: list[int], operators: list[Operator]) -> int:
        accum = numbers[0]
        for operator, number in zip(operators, numbers[1:]):
            accum = operator(accum, number)
        return accum

    def can_be_true(self) -> bool:
        combinations = product(self.op_types, repeat=len(self.numbers) - 1)
        for combination in combinations:
            value = self.evaluate_l2r(self.numbers, combination)
            if value == self.value:
                return True
        return False

    @classmethod
    def from_string(
        cls, line: str, op_types: list[Type[Operator]], separator: str = ":"
    ) -> "Equation":
        value, numbers = line.split(sep=separator)
        return cls(
            value=int(value),
            numbers=[int(n) for n in numbers.split()],
            op_types=op_types,
        )


def parse_equations(filepath: str, op_types: list[Type[Operator]]):
    with open(filepath, "r", encoding="utf-8") as f:
        equations = [Equation.from_string(line, op_types) for line in f.readlines()]
    return equations

In [52]:
equations = parse_equations("./example.txt", [Add(), Multiply()])
sum(equation.value for equation in equations if equation.can_be_true())

3749

In [54]:
equations = parse_equations("./input.txt", [Add(), Multiply()])
sum(equation.value for equation in equations if equation.can_be_true())

303876485655

# Part Two

The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a third type of operator.

The **concatenation operator** (`||`) combines the digits from its left and right inputs into a single number. For example:

- `12 || 345` would become `12345`.

All operators are still evaluated **left-to-right**.

Now, apart from the three equations that could be made true using only addition (`+`) and multiplication (`*`), the example has **three more equations** that can be made true by inserting the concatenation operator (`||`):

1. **156**: `15 6` can be made true through a single concatenation:  
   `15 || 6 = 156`.

2. **7290**: `6 8 6 15` can be made true using:  
   `6 * 8 || 6 * 15`.

3. **192**: `17 8 14` can be made true using:  
   `17 || 8 + 14`.

Adding up **all six test values** (the three that could be made before using only `+` and `*`, plus the new three that can now be made by also using `||`) produces the new total calibration result of **11387**.

---

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. **What is their total calibration result?**

In [55]:
class Concatenation(Operator):
    def __call__(self, a: int, b: int) -> int:
        return int("".join(map(str, [a, b])))


equations = parse_equations("./example.txt", [Add(), Multiply(), Concatenation()])
sum(equation.value for equation in equations if equation.can_be_true())

11387

In [56]:
equations = parse_equations("./input.txt", [Add(), Multiply(), Concatenation()])
sum(equation.value for equation in equations if equation.can_be_true())

146111650210682